# Import thư viện 

In [1]:
import numpy as np 
import pandas as pd 
import tmdbv3api as tmdb 

In [2]:
from tmdbv3api import TMDb, Movie, Search
from os import getenv
tmdb = TMDb()

tmdb.api_key = getenv('TMDB_API_KEY')
search = Search()

# New form

In [3]:
from typing import List

"""
- Hàm lấy tên, ngày film từ wiki

"""
def get_films_on_wiki(year: int, prelink: str, title_label: str): 
    link = prelink + str(year)

    data = pd.read_html(link)
    if len(data) < 6: return None
    sub_df = pd.concat([data[2], data[3], data[4], data[5]])
  
    res = sub_df[['Opening', title_label, "Opening.1"]].copy()
    res.dropna(inplace=True)
    res.rename(columns={"Opening":"Month",title_label: "English title", "Opening.1": "Day"}, inplace=True)
    return res


- Sử lại Opening: lower_case + remove space + chuyển sang int 
- Kết hợp Opening.1 + Opening

In [4]:
dct = {
    'january' : 1, 
    'february' : 2,
    'march' : 3, 
    'april' :  4,
    'may' : 5, 
    'june' : 6, 
    'july' : 7, 
    'august' : 8, 
    'september' : 9, 
    'october' : 10, 
    'november' : 11, 
    'december' : 12
}

def combine_day_with_month(x: tuple, year : int):
    return  str(year) + "-" + str(x[0]) + "-" + str(x[1])

def month_to_int(month : str) : 
    if month not in  dct.keys():
        return None 
    return dct[month]

def released_handling(year: int, df: pd.DataFrame):
   
    df['Month'] = df['Month'].str.lower()
    df['Month'] = df['Month'].str.replace(' ', "")

    df['Month']  = df['Month'].apply(lambda x: month_to_int(x)) #con
    
    df['Day'] = df['Day'].astype(int)
    df['Released'] = df[['Month', 'Day']].apply(lambda x: combine_day_with_month(x, year), axis=1)
    df['Released'] = pd.to_datetime(df['Released'])
    df.drop(['Month', 'Day'], inplace = True, axis=1)
    return df

    pass

In [5]:
def get_tmdb_id(title, release_date):
    if release_date is None: return None
    t  = release_date.year
    movie = search.movies(title, release_year=t)

    for j in movie['results']: 
        if j['release_date'] == release_date.strftime('%Y-%m-%d'): 
            return j['id']
        
    return None

In [15]:
def gather_data(country: str, year_start: int, year_end): 
    prelink = 'https://en.wikipedia.org/wiki/List_of_' + country + "_films_of_"
    data_set = pd.DataFrame(columns=['Released', 'English title', 'TMDB_id']) # create
    for i in range(year_start, year_end + 1): 
        print(f'Year: {i}')
        compo_df = get_films_on_wiki(i, prelink, "Title")
        
        if compo_df is None: continue
        compo_clean = released_handling(df= compo_df, year=i)

        compo_clean['TMDB_id'] = compo_clean[['English title', 'Released']].apply(lambda x: get_tmdb_id(x[0], x[1]), axis = 1)

        data_set = pd.concat([data_set, compo_clean])
        
        
    return data_set

In [17]:

t = gather_data("Japanese",2014, 2023)
t.dropna(inplace=True)
t['TMDB_id'] = t['TMDB_id'].astype(int)
t.reset_index(inplace=True)
path = 'Data/raw_jap_12_23.csv'
t.to_csv(path)

Year: 2014


C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\2035501406.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return  str(year) + "-" + str(x[0]) + "-" + str(x[1])
C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\3868877511.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  compo_clean['TMDB_id'] = compo_clean[['English title', 'Released']].apply(lambda x: get_tmdb_id(x[0], x[1]), axis = 1)
C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\3868877511.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or a

Year: 2015


C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\2035501406.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return  str(year) + "-" + str(x[0]) + "-" + str(x[1])
C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\3868877511.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  compo_clean['TMDB_id'] = compo_clean[['English title', 'Released']].apply(lambda x: get_tmdb_id(x[0], x[1]), axis = 1)


Year: 2016


...

C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\2035501406.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return  str(year) + "-" + str(x[0]) + "-" + str(x[1])
C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\3868877511.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  compo_clean['TMDB_id'] = compo_clean[['English title', 'Released']].apply(lambda x: get_tmdb_id(x[0], x[1]), axis = 1)


Year: 2019


C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\2035501406.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return  str(year) + "-" + str(x[0]) + "-" + str(x[1])
C:\Users\Asus\AppData\Local\Temp\ipykernel_44048\3868877511.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  compo_clean['TMDB_id'] = compo_clean[['English title', 'Released']].apply(lambda x: get_tmdb_id(x[0], x[1]), axis = 1)


Year: 2020


ValueError: invalid literal for int() with base 10: 'none'

In [12]:
t

,index,Released,English title,TMDB_id
0,0,2020-01-02,Dogs in the House,658392
1,1,2020-01-09,Don't Cry for Me Sudan: Shukran Baba,652805
2,2,2020-01-09,Montmartre de Papa,653075
3,3,2020-01-15,Secret Zoo,575718
4,4,2020-01-16,Gang,636293
...,...,...,...,...
250,13,2023-11-22,12.12: The Day,919207
251,15,2023-11-29,Single in Seoul,717088
252,16,2023-12-06,Our Season,661681
253,17,2023-12-06,A Letter from Kyoto,1021633
